<a href="https://colab.research.google.com/github/BoonyaweePairee/Front-End-Node/blob/main/scrap_data_HW.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install bs4

In [ ]:
pip install requests


In [ ]:
pip install pandas

In [ ]:
pip install openpyxl

In [ ]:
pip install azure-storage-blob

In [ ]:
pip install beautifulsoup4

In [ ]:
pip install songline

  Preparing metadata (setup.py) ... done
  Created wheel for songline: filename=songline-0.0.2-py3-none-any.whl size=4868 sha256=325c07df13378e8db8f6df5c59fb44a263037ccd56d737260fd6af8169cf99b5
  Stored in directory: /root/.cache/pip/wheels/b2/5c/85/9a7ba890449b322545ab4628d052b91155c6379b4f9a8669f4
Successfully built songline


In [ ]:
import pandas as pd
import bs4
import requests
import openpyxl
import os
from urllib.request import urlopen as req
from bs4 import BeautifulSoup as Soup
from multiprocessing.pool import ThreadPool
from azure.storage.blob import BlobServiceClient, BlobClient
from azure.storage.blob import ContentSettings, ContainerClient

In [ ]:
item = 1
name_list = []
price_list = []

while item <= 20 :
  data = requests.get('https://www.redwolfairsoft.com/guns/all?p=' + str(item))
  soup = bs4.BeautifulSoup(data.text)

  name = soup.find_all('strong',{'class':'product name product-item-name'})
  price = soup.find_all('span',{'class':'price'}, limit = 25)

  for i in name:
    name_list.append(i.text)

  for i in price:
    price_list.append(i.text)

  print('Complete item number: ',item)
  item += 1

print(name_list)
print(price_list)
table = pd.DataFrame([name_list,price_list]).transpose()
table.columns = ['name','price']
table.set_index('name')
table.to_excel('AirsoftGunSell.xlsx',engine='openpyxl')

Complete item number:  1
Complete item number:  2
Complete item number:  3
Complete item number:  4
Complete item number:  5
Complete item number:  6
Complete item number:  7
Complete item number:  8
Complete item number:  9
Complete item number:  10
Complete item number:  11
Complete item number:  12
Complete item number:  13
Complete item number:  14
Complete item number:  15
Complete item number:  16
Complete item number:  17
Complete item number:  18
Complete item number:  19
Complete item number:  20
['SIG Sauer MPX-K Sportline Airsoft AEG Rifle (by SIG AIR & King Arms) - Black', 'Viva Arms Mosin Nagant M44 Carbine (Real Wood CO2 Version)', 'RWA Nighthawk Custom War Hawk Green Gas Airsoft Pistol (Rail Version) - Cerakote COBALT', 'Army Armament John Wick 4 Taran Tactical Pit Viper GBB Airsoft Pistol (R614-1) (Standard Version)', 'EMG Staccato Licensed XC 2011 GBB Airsoft Pistol (Model: VIP Grip / Standard / Green Gas)', 'WELL PRO M134 Mini Vulcan Rotating Gatling Airsoft AEG - Bla

In [ ]:
storage_account_key = "a+xMAKZuhH59jmx6EIeYhnNADM2sO441qPG7GE29/VTR+svZdYh4KmK+RZfwtZY6woAcMjN5eJaB+ASt26ecRA=="
storage_account_name = "boonyaweebigdata1"
connection_string = "DefaultEndpointsProtocol=https;AccountName=boonyaweebigdata1;AccountKey=a+xMAKZuhH59jmx6EIeYhnNADM2sO441qPG7GE29/VTR+svZdYh4KmK+RZfwtZY6woAcMjN5eJaB+ASt26ecRA==;EndpointSuffix=core.windows.net"
container_name = "blobinsertdata"

def uploadToBlobStorage(file_path,file_name):
  blob_service_client = BlobServiceClient.from_connection_string(connection_string)
  blob_client = blob_service_client.get_blob_client(container=container_name,blob=file_name)
  try:
    if blob_client.exists():
        # Delete blob
        blob_client.delete_blob()
        print("Blob deleted successfully!")
    else:
        print("Blob does not exist!")
    with open(file_path,"rb") as data:
      blob_client.upload_blob(data)
    print(f"Uploaded {file_name}.successfully!")
  except Exception as e:
    print("Failed to upload files to container. Error:" + str(e))

uploadToBlobStorage('/content/AirsoftGunSell.xlsx','AirsoftGunSell.xlsx')

Blob does not exist!
Uploaded AirsoftGunSell.xlsx.successfully!


In [ ]:
import os
from multiprocessing.pool import ThreadPool
from azure.storage.blob import BlobServiceClient, BlobClient
from azure.storage.blob import ContentSettings, ContainerClient
import pandas as pd

In [ ]:
MY_CONNECTION_STRING = "DefaultEndpointsProtocol=https;AccountName=boonyaweebigdata1;AccountKey=a+xMAKZuhH59jmx6EIeYhnNADM2sO441qPG7GE29/VTR+svZdYh4KmK+RZfwtZY6woAcMjN5eJaB+ASt26ecRA==;EndpointSuffix=core.windows.net"
MY_BLOB_CONTAINER = "blobinsertdata"

LOCAL_BLOB_PATH = "/content/download_data"

class AzureBlobFileDownloader:
  def __init__(self):
    print("Intializing AzureBlobFileDownloader")

    self.blob_service_client =  BlobServiceClient.from_connection_string(MY_CONNECTION_STRING)
    self.my_container = self.blob_service_client.get_container_client(MY_BLOB_CONTAINER)

  def download_all_blobs_in_container(self):
    my_blobs = self.my_container.list_blobs()
    result = self.run(my_blobs)
    print(result)

  def run(self,blobs):
    with ThreadPool(processes=int(10)) as pool:
     return pool.map(self.save_blob_locally, blobs)

  def save_blob_locally(self,blob):
    file_name = blob.name
    print(file_name)
    bytes = self.my_container.get_blob_client(blob).download_blob().readall()

    download_file_path = os.path.join(LOCAL_BLOB_PATH, file_name)
    os.makedirs(os.path.dirname(download_file_path), exist_ok=True)

    with open(download_file_path, "wb") as file:
      file.write(bytes)
    return file_name

azure_blob_file_downloader = AzureBlobFileDownloader()
azure_blob_file_downloader.download_all_blobs_in_container()

dataframe1 = pd.read_excel('/content/download_data/AirsoftGunSell.xlsx')
print(dataframe1)


Intializing AzureBlobFileDownloader
AirsoftGunSell.xlsx
stockblob.xlsx
['AirsoftGunSell.xlsx', 'stockblob.xlsx']
     Unnamed: 0                                               name      price
0             0  SIG Sauer MPX-K Sportline Airsoft AEG Rifle (b...    $189.99
1             1  Viva Arms Mosin Nagant M44 Carbine (Real Wood ...    $359.99
2             2  RWA Nighthawk Custom War Hawk Green Gas Airsof...    $249.99
3             3  Army Armament John Wick 4 Taran Tactical Pit V...    $104.99
4             4  EMG Staccato Licensed XC 2011 GBB Airsoft Pist...    $139.99
..          ...                                                ...        ...
495         495  Tokyo Marui Type 89 J.G.S.D.F. GBBR Airsoft (Z...    $509.99
496         496                   G&G ARP556 AEG - Battleship Grey    $329.99
497         497                            LCT PKP Airsoft LMG AEG    $949.99
498         498  Airsoft Surgeon Airsoft Revolver (Black Hawk S...  $3,206.00
499         499  AW Custom HX